## Merges the tweets database with the related events ids

In [1]:
import pandas as pd
from datetime import datetime as DateTime, timedelta as TimeDelta, date as Date
from tqdm import tqdm, tqdm_notebook
pd.options.display.max_colwidth = 250
pd.__version__ # marche pas avec pandas 0.25 https://github.com/tqdm/tqdm/issues/780
tqdm_notebook().pandas()

'0.24.2'

In [2]:
INPUT_FILE_GZIP = './_sources_final/pickles/pression_sociale_rounds_1_2_3_corrected_search_str_opt_clean_text_dup100_clean_matches.pickle.gzip'

In [3]:
df = pd.read_pickle(INPUT_FILE_GZIP, compression='gzip')

In [4]:
len(df)

15882669

In [5]:
df.head(2)

,id,conversation_id,date_end,date_start,datetime,is_quote,is_reply,is_retweet,nbr_favorite,nbr_reply,...,search_string,text,user_id,user_name,string_date,text_clean,dup_100,str_in_text,str_in_user,str_in_conv
0,940606059626090496,940606059626090496,2018-02-25,2017-12-11,2017-12-12 16:35:34,False,False,False,3,0,...,Accord Healthcare,We talk exclusively to Accord Healthcare 's EU Supply Operations Director about the firms #supplychain competence and agility in the crowded #healthcare market http:// bit.ly/2z3W22g pic.twitter.com/jKuLRbUQqT,60939437,SupplyChainD,accordhealthcare_2017-12-11,we talk exclusively to accord healthcare s eu supply operations director about the firms supply chain competence and agility in the crowded healthcare market,False,True,False,False
1,940254856471896064,940254856471896064,2018-02-25,2017-12-11,2017-12-11 17:20:01,False,False,False,0,0,...,Accord Healthcare,"""The equitable/legal title conundrum and claiming priority in #patent applications"" \n\nRead our article following the Accord Healthcare Limited v Research Corporation Technologies Inc, 2017 decision: http:// bit.ly/2yeZfrT \n\n#priority ...",498639837,VennerShipley,accordhealthcare_2017-12-11,the equitable legal title conundrum and claiming priority in patent applications read our article following the accord healthcare limited v research corporation technologies inc decision priority,False,True,False,False


In [6]:
controls = pd.read_csv('./_sources_final/control.csv')

In [7]:
controls.head(2)

,nid,announcement_date,search_string,done,date_iso,date_start,date_end,Unnamed: 7
0,76423,06/01/2014,Sisters Food,Done,2014-01-06,2013-11-22,2014-02-06,NaN
1,78656,20/02/2015,Sisters Food,Done,2015-02-20,2015-01-06,2015-03-23,NaN


In [8]:
def date_iso(date):
    date = date.split('/')
    return '{}-{}-{}'.format(date[2], date[1], date[0])

In [9]:
JOURS_AVANT = 45
def start_date(date):
    date = DateTime.strptime(date, "%Y-%m-%d") - TimeDelta(days=JOURS_AVANT)
    return date.strftime("%Y-%m-%d")

In [10]:
JOURS_APRES = 30
def end_date(date):
    date = DateTime.strptime(date, "%Y-%m-%d") + TimeDelta(days=JOURS_APRES + 1)
    return date.strftime("%Y-%m-%d")

In [11]:
clean_string = lambda x: x.replace(' ', '').lower()
string_date = lambda x, y: '{}_{}'.format(clean_string(x), clean_string(y))

In [12]:
controls['date_iso'] = controls.announcement_date.apply(date_iso)

In [13]:
controls['date_start'] = controls.date_iso.apply(start_date)

In [14]:
controls['date_end'] = controls.date_iso.apply(end_date)

In [15]:
controls['string_date'] = controls.apply(lambda row: string_date(row['search_string'], row['date_start']), axis=1)

In [16]:
controls.drop(['Unnamed: 7'], axis=1, inplace=True)

In [17]:
controls.head()

,nid,announcement_date,search_string,done,date_iso,date_start,date_end,string_date
0,76423,06/01/2014,Sisters Food,Done,2014-01-06,2013-11-22,2014-02-06,sistersfood_2013-11-22
1,78656,20/02/2015,Sisters Food,Done,2015-02-20,2015-01-06,2015-03-23,sistersfood_2015-01-06
2,89055,01/11/2016,Sisters Food,Done,2016-11-01,2016-09-17,2016-12-02,sistersfood_2016-09-17
3,89795,06/01/2017,Sisters Food,Done,2017-01-06,2016-11-22,2017-02-06,sistersfood_2016-11-22
4,90847,07/04/2017,Sisters Food,Done,2017-04-07,2017-02-21,2017-05-08,sistersfood_2017-02-21


In [18]:
indexes = dict()

In [19]:
for index, row in controls.iterrows():
    indexes[row['string_date']] = row['nid']

In [20]:
idx = lambda x: indexes[x]

In [21]:
df.loc[:,'nid'] = df.string_date.progress_apply(idx)

In [22]:
df.head(2)

,id,conversation_id,date_end,date_start,datetime,is_quote,is_reply,is_retweet,nbr_favorite,nbr_reply,...,text,user_id,user_name,string_date,text_clean,dup_100,str_in_text,str_in_user,str_in_conv,nid
0,940606059626090496,940606059626090496,2018-02-25,2017-12-11,2017-12-12 16:35:34,False,False,False,3,0,...,We talk exclusively to Accord Healthcare 's EU Supply Operations Director about the firms #supplychain competence and agility in the crowded #healthcare market http:// bit.ly/2z3W22g pic.twitter.com/jKuLRbUQqT,60939437,SupplyChainD,accordhealthcare_2017-12-11,we talk exclusively to accord healthcare s eu supply operations director about the firms supply chain competence and agility in the crowded healthcare market,False,True,False,False,93132
1,940254856471896064,940254856471896064,2018-02-25,2017-12-11,2017-12-11 17:20:01,False,False,False,0,0,...,"""The equitable/legal title conundrum and claiming priority in #patent applications"" \n\nRead our article following the Accord Healthcare Limited v Research Corporation Technologies Inc, 2017 decision: http:// bit.ly/2yeZfrT \n\n#priority ...",498639837,VennerShipley,accordhealthcare_2017-12-11,the equitable legal title conundrum and claiming priority in patent applications read our article following the accord healthcare limited v research corporation technologies inc decision priority,False,True,False,False,93132


In [23]:
OUTPUT_FILE_GZIP = './_sources_final/pickles/pression_sociale_rounds_1_2_3_corrected_search_str_opt_clean_text_dup100_clean_matches_nids.pickle.gzip'

In [24]:
df.to_pickle(OUTPUT_FILE_GZIP, compression='gzip')

#### Export to CSV

In [25]:
df.to_csv('./_sources_final/outputs/social_pressure_2019_08_08.csv', compression='gzip')

In [26]:
df.dtypes

id                  int64
conversation_id     int64
date_end           object
date_start         object
datetime           object
is_quote             bool
is_reply             bool
is_retweet           bool
nbr_favorite        int64
nbr_reply           int64
nbr_retweet         int64
search_string      object
text               object
user_id             int64
user_name          object
string_date        object
text_clean         object
dup_100              bool
str_in_text          bool
str_in_user          bool
str_in_conv          bool
nid                 int64
dtype: object

In [27]:
df.drop(['text', 'search_string', 'text_clean'], axis=1, inplace=True)

In [28]:
df.to_csv('./_sources_final/outputs/social_pressure_no_text_2019_08_08.csv', compression='gzip')

### Counts by nid

In [29]:
counts = df.nid.value_counts().to_frame().reset_index()
counts.head(2)

,index,nid
0,74758,506051
1,88614,504564


In [30]:
counts.columns = ['nid', 'total']
counts.head(2)

,nid,total
0,74758,506051
1,88614,504564


In [31]:
counts_dict = dict()
for index, row in counts.iterrows():
    counts_dict[row.nid] = row.total

In [32]:
def get_counts(x):
    try:
        out = counts_dict[x]
    except:
        out = 0
    return out

In [33]:
controls['nb_tweets'] = controls.nid.apply(get_counts)

In [34]:
controls.head(2)

,nid,announcement_date,search_string,done,date_iso,date_start,date_end,string_date,nb_tweets
0,76423,06/01/2014,Sisters Food,Done,2014-01-06,2013-11-22,2014-02-06,sistersfood_2013-11-22,542
1,78656,20/02/2015,Sisters Food,Done,2015-02-20,2015-01-06,2015-03-23,sistersfood_2015-01-06,328


In [35]:
controls.to_excel('./_sources_final/controls/05_scraping_conntrols.xlsx')